# Introduction - Spark DataFrames API

In this notebook exercise, we are going to mainly focus on understanding how DataFrames API work and how spark internals work in general. 

We start with reading data using the DataSources API and walking through different parts of the DataFrame API to demonstrate how it can be used to process big data. 

Later we will move to more advanced operations in the DataFrames API and then dive into some internals and the problem of data skew.

## Spark Session 

In Apache Spark 2.0 and later versions, Spark introduced a Context variable that belongs to `SparkSession` class type.

In earlier versions of Apach Spark, There were two main context variables available.
1. `SparkContext` that enabled RDD API operations
2. `SQLContext` that enables DataFrame API and SparkSQL operations

In Spark 2.0 and above versions, Apache Spark officially introduced a `SparkSession` Class that allows the user to keep both Spark Context and SQL Context in the same context variable simplifying the developer experience.

In [47]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [48]:
spark.sparkContext # RDD API entrypoint

<SparkContext master=local[*] appName=pyspark-shell>

## Reading data using Data Sources

DataFrames API uses datasources to read files from distributed/non-distributed data sources. Let's read some JSON string records to start with. Spark also allows using POSIX wildcards when expressing a collection of filepaths.

In [49]:
filepath='D:/DataScienceTraining/ADS06/03-spark_dataframes/data/'
reviews_paths =[filepath+'reviews_fashion.json', 
                filepath+'reviews_electronics.json',
                filepath+'reviews_sports.json']

print(reviews_paths)

['D:/DataScienceTraining/ADS06/03-spark_dataframes/data/reviews_fashion.json', 'D:/DataScienceTraining/ADS06/03-spark_dataframes/data/reviews_electronics.json', 'D:/DataScienceTraining/ADS06/03-spark_dataframes/data/reviews_sports.json']


In [14]:
inferred_reviews = spark.read.json(reviews_paths)
inferred_reviews.take(2)

[Row(asin='0000031887', helpful=[0, 0], overall=5.0, reviewText='Perfect red tutu for the price. I baught it as part of my daughters Halloween costume and it looked great on her.', reviewTime='11 4, 2013', reviewerID='A2XVJBSRI3SWDI', reviewerName='abigail', summary='Nice tutu', unixReviewTime=1383523200),
 Row(asin='0000031887', helpful=[1, 1], overall=4.0, reviewText='This was a really cute tutu the only problem is that it was super short on my 5 yr old daughter. Other than that it was really adorable.', reviewTime='05 26, 2012', reviewerID='A2G0LNLN79Q6HR', reviewerName='aj_18 "Aj_18"', summary='Really Cute but rather short.', unixReviewTime=1337990400)]

### Exercise

Define a POSIX representation for the above 3 files and read the reviews data to verify Spark reads 30,000 records. (5 mins)

Hint: [POSIX Cheatsheet](https://en.wikibooks.org/wiki/Regular_Expressions/POSIX_Basic_Regular_Expressions)

In [4]:
# Initialize a variable and call it "POSIX". The assign it with a string value representing 
# the filepath pattern that will capture all three .json files

POSIX = filepath+"reviews_*.json" 


In [5]:
inferred_reviews = spark.read.json(POSIX)

In [7]:
inferred_reviews.take(2)

[Row(asin='0132793040', helpful=[1, 1], overall=5.0, reviewText='Corey Barker does a great job of explaining Blend Modes in this DVD. All of the Kelby training videos are great but pricey to buy individually. If you really want bang for your buck just subscribe to Kelby Training online.', reviewTime='04 13, 2013', reviewerID='AKM1MP6P0OYPR', reviewerName='Vicki Gibson "momo4"', summary='Very thorough', unixReviewTime=1365811200),
 Row(asin='0321732944', helpful=[0, 0], overall=5.0, reviewText="While many beginner DVDs try to teach you everything there is to know about Photoshop CS5, this introductory course shows you the critical things you need to know to start feeling confident in your Photoshop skills. Bestselling author and Photoshop trainer, Matt Kloskowski shows you exactly what you need to know about tools, filters, adjustments, palettes, and menu items to hit the ground running in Photoshop. These include: layers and how they're key to mastering Photoshop, selections and the es

### Binding data to a Schema

From the above example, it is seen that we did not have to define a schema for the reviews dataset. Apache Spark automatically scans through the files and *infers* the schema of the dataset.

#### Explicitly defining schema

In certain cases, it is necessary for the schema to be defined explicitly. In instances such as:
- missing data
- potential future scenarios
- explicit business constrains

In [8]:
# Export the modules
from pyspark.sql.types import *

# Define Schema
REVIEWS_SCHEMA_DEF = StructType([
        StructField('reviewerID', StringType(), True),
        StructField('asin', StringType(), True),
        StructField('reviewerName', StringType(), True),
        StructField('helpful', ArrayType(
                IntegerType(), True), 
            True),
#        review text column definition missing here
        StructField('reviewTime', StringType(), True),
        StructField('overall', DoubleType(), True),
        StructField('summary', StringType(), True),
        StructField('unixReviewTime', LongType(), True)
    ])

print(REVIEWS_SCHEMA_DEF)

StructType(List(StructField(reviewerID,StringType,true),StructField(asin,StringType,true),StructField(reviewerName,StringType,true),StructField(helpful,ArrayType(IntegerType,true),true),StructField(reviewTime,StringType,true),StructField(overall,DoubleType,true),StructField(summary,StringType,true),StructField(unixReviewTime,LongType,true)))


The `reviewText` field is not added to the schema here. What do you think will happen if we try to enforce this schema to the `reviews` dataset?
1. Spark will enforce manual schema on the defined columns and ignore others
2. Spark will enforce manual schema on the defined columns and infer schema for others
3. Spark will fail with a SchemaMismatch exception

In [9]:
# Try it out
reviews = spark.read.json(POSIX, schema=REVIEWS_SCHEMA_DEF)
print("The answer is {}".format(reviews))

The answer is DataFrame[reviewerID: string, asin: string, reviewerName: string, helpful: array<int>, reviewTime: string, overall: double, summary: string, unixReviewTime: bigint]


In [10]:
reviews.printSchema()

root
 |-- reviewerID: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- reviewTime: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



### Exercise
Add `reviewText` field to the data schema (5 mins)

In [11]:
# Define FULL_REVIEWS_SCHEMA_DEF here

FULL_REVIEWS_SCHEMA_DEF = StructType([
        StructField('reviewerID', StringType(), True),
        StructField('asin', StringType(), True),
        StructField('reviewerName', StringType(), True),
        StructField('helpful', ArrayType(
                IntegerType(), True), 
            True),
        StructField('reviewText', StringType(), True),
        StructField('reviewTime', StringType(), True),
        StructField('overall', DoubleType(), True),
        StructField('summary', StringType(), True),
        StructField('unixReviewTime', LongType(), True)
    ])

print(FULL_REVIEWS_SCHEMA_DEF)

StructType(List(StructField(reviewerID,StringType,true),StructField(asin,StringType,true),StructField(reviewerName,StringType,true),StructField(helpful,ArrayType(IntegerType,true),true),StructField(reviewText,StringType,true),StructField(reviewTime,StringType,true),StructField(overall,DoubleType,true),StructField(summary,StringType,true),StructField(unixReviewTime,LongType,true)))


In [12]:
reviews = spark.read.json(POSIX, schema=FULL_REVIEWS_SCHEMA_DEF)
reviews.printSchema()

root
 |-- reviewerID: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



# DataFrame operations

Spark DataFrame API allow you to do multiple operations on the Data. The primary advantage of using the DataFrame API is that you can do data transformations with the high level API without having to use Python. Using the high level API has performance advantages.

DataFrame API have functionality similar to that of Core RDD API. For example: 
+ `map`                       : `foreach`, `Select`
+ `filter`                    : `filter`
+ `groupByKey`, `reduceByKey` : `groupBy`

## Selecting Columns

You can use SELECT statement to select columns from your dataframe

In [13]:
select_df = reviews.select(reviews.asin,
                           reviews.overall,
                           reviews.reviewText,
                           reviews.helpful[0]/reviews.helpful[1],
                           reviews.reviewerID,
                           reviews.unixReviewTime).\
                    withColumnRenamed('(helpful[0] / helpful[1])','helpful')

select_df.show()

+----------+-------+--------------------+------------------+--------------+--------------+
|      asin|overall|          reviewText|           helpful|    reviewerID|unixReviewTime|
+----------+-------+--------------------+------------------+--------------+--------------+
|0132793040|    5.0|Corey Barker does...|               1.0| AKM1MP6P0OYPR|    1365811200|
|0321732944|    5.0|While many beginn...|              null|A2CX7LUOHB2NDG|    1341100800|
|0439886341|    1.0|It never worked. ...|               1.0|A2NWSAGRHCP8N5|    1367193600|
|0439886341|    3.0|Some of the funct...|               1.0|A2WNBOD3WNDNKT|    1374451200|
|0439886341|    1.0|Do not waste your...|               1.0|A1GI0U4ZRJA8WN|    1334707200|
|0511189877|    5.0|Dog got the old r...|              null|A1QGNMC6O1VW39|    1397433600|
|0511189877|    2.0|This remote, for ...|               1.0|A3J3BRHTDRFJ2G|    1397433600|
|0511189877|    5.0|We had an old Tim...|               0.0|A2TY0BTJOTENPG|    1395878400|

## Filtering rows

Filtering lets you select rows based on arguments. The implementation pattern is similar to filtering RDDs, But simpler. Let's filter the reviews that have got an overall score of 3 or more... 

In [14]:
filter_df = select_df.filter(select_df.overall >= 3)
filter_df.show()

+----------+-------+--------------------+------------------+--------------+--------------+
|      asin|overall|          reviewText|           helpful|    reviewerID|unixReviewTime|
+----------+-------+--------------------+------------------+--------------+--------------+
|0132793040|    5.0|Corey Barker does...|               1.0| AKM1MP6P0OYPR|    1365811200|
|0321732944|    5.0|While many beginn...|              null|A2CX7LUOHB2NDG|    1341100800|
|0439886341|    3.0|Some of the funct...|               1.0|A2WNBOD3WNDNKT|    1374451200|
|0511189877|    5.0|Dog got the old r...|              null|A1QGNMC6O1VW39|    1397433600|
|0511189877|    5.0|We had an old Tim...|               0.0|A2TY0BTJOTENPG|    1395878400|
|0511189877|    5.0|This unit works j...|              null|A34ATBPOK6HCHY|    1395532800|
|0511189877|    5.0|It is an exact du...|              null| A89DO69P0XZ27|    1395446400|
|0511189877|    5.0|Works on my t.v. ...|               0.0| AZYNQZ94U6VDB|    1401321600|

### Exercise

Use the `where` function to filter out rows from `select_df` to get a list of rows *where* `helpful` score is more than or equal to 0.5 (5 mins)

In [15]:
filter_df = select_df.where(select_df.helpful >= 0.5)
filter_df.show()


+----------+-------+--------------------+------------------+--------------+--------------+
|      asin|overall|          reviewText|           helpful|    reviewerID|unixReviewTime|
+----------+-------+--------------------+------------------+--------------+--------------+
|0132793040|    5.0|Corey Barker does...|               1.0| AKM1MP6P0OYPR|    1365811200|
|0439886341|    1.0|It never worked. ...|               1.0|A2NWSAGRHCP8N5|    1367193600|
|0439886341|    3.0|Some of the funct...|               1.0|A2WNBOD3WNDNKT|    1374451200|
|0439886341|    1.0|Do not waste your...|               1.0|A1GI0U4ZRJA8WN|    1334707200|
|0511189877|    2.0|This remote, for ...|               1.0|A3J3BRHTDRFJ2G|    1397433600|
|0528881469|    1.0|I'm a professiona...|               0.8| AMO214LNFCEI4|    1290643200|
|0528881469|    4.0|This is a great t...|0.9545454545454546|A28B1G1MSJ6OO1|    1280016000|
|0528881469|    3.0|Well, what can I ...|0.9555555555555556|A3N7T0DY83Y4IG|    1283990400|

## Grouping by overall scores

Grouping is equivalent to the groupByKey in the core RDD API. You can transform the grouped values using a summary action such as:
+ count
+ sum
+ average
+ max and so on ...


Lets group the number of reviews by the overall score in the reviews dataset to see the number of entries each score has received. 

In [22]:
grouped = select_df.groupBy([select_df.overall, select_df.helpful]).count().orderBy(['overall'])
grouped.show()

+-------+-------------------+-----+
|overall|            helpful|count|
+-------+-------------------+-----+
|    1.0|              0.125|    7|
|    1.0|               0.75|   53|
|    1.0|0.25925925925925924|    1|
|    1.0| 0.5384615384615384|    3|
|    1.0| 0.9047619047619048|    2|
|    1.0|0.34782608695652173|    1|
|    1.0| 0.2631578947368421|    1|
|    1.0|0.16666666666666666|   11|
|    1.0|               0.95|    3|
|    1.0| 0.2857142857142857|    5|
|    1.0| 0.4090909090909091|    2|
|    1.0|0.14634146341463414|    1|
|    1.0| 0.9333333333333333|    5|
|    1.0| 0.9545454545454546|    1|
|    1.0| 0.8253968253968254|    1|
|    1.0| 0.9285714285714286|    1|
|    1.0| 0.7083333333333334|    1|
|    1.0| 0.5217391304347826|    1|
|    1.0| 0.8135593220338984|    1|
|    1.0|0.23076923076923078|    1|
+-------+-------------------+-----+
only showing top 20 rows



## Joining DataFrames together

You can join two DataFrames together by using a common key. Two different datasets can be used to perform a join. 

Let's look at the second dataset that stores product details before we dive into joining.

In [31]:
# Load Dataset2 : Amazon Product information
product_filepaths = filepath+'products_*.json'

# First, define Schema for second Dataset
PRODUCTS_SCHEMA_DEF = StructType([
        StructField('asin', StringType(), True),
        StructField('title', StringType(), True),
        StructField('price', DoubleType(), True),
        StructField('categories', ArrayType(ArrayType(
            StringType(), True),True),True)
    ])

# Load the dataset
product_df = spark.read.json(product_filepaths, PRODUCTS_SCHEMA_DEF)

product_df.select('categories').show()

+--------------------+
|          categories|
+--------------------+
|[[Clothing, Shoes...|
|[[Sports & Outdoo...|
|[[Sports & Outdoo...|
|[[Sports & Outdoo...|
|[[Sports & Outdoo...|
|[[Sports & Outdoo...|
|[[Clothing, Shoes...|
|[[Sports & Outdoo...|
|[[Clothing, Shoes...|
|[[Electronics, Co...|
|[[Sports & Outdoo...|
|[[Electronics, Co...|
|[[Electronics, Co...|
|[[Clothing, Shoes...|
|[[Clothing, Shoes...|
|[[Clothing, Shoes...|
|[[Clothing, Shoes...|
|[[Clothing, Shoes...|
|[[Clothing, Shoes...|
|[[Clothing, Shoes...|
+--------------------+
only showing top 20 rows



Now let us join the two datasets together based on `asin`. Let's connect all the reviews available to the product details found in the products dataframe.

In [33]:
product_reviews = (filter_df.join(product_df, 
                                  product_df.asin == filter_df.asin).
                              dropna(subset="title"))
product_reviews.show()

+----------+-------+--------------------+------------------+--------------+--------------+----------+--------------------+------+--------------------+
|      asin|overall|          reviewText|           helpful|    reviewerID|unixReviewTime|      asin|               title| price|          categories|
+----------+-------+--------------------+------------------+--------------+--------------+----------+--------------------+------+--------------------+
|0132793040|    5.0|Corey Barker does...|               1.0| AKM1MP6P0OYPR|    1365811200|0132793040|Kelby Training DV...|  null|[[Electronics, Co...|
|0439886341|    1.0|It never worked. ...|               1.0|A2NWSAGRHCP8N5|    1367193600|0439886341|Digital Organizer...|  8.15|[[Electronics, Co...|
|0439886341|    3.0|Some of the funct...|               1.0|A2WNBOD3WNDNKT|    1374451200|0439886341|Digital Organizer...|  8.15|[[Electronics, Co...|
|0439886341|    1.0|Do not waste your...|               1.0|A1GI0U4ZRJA8WN|    1334707200|0439

When you join two RDDs, you have to reshape the data into (k,v) pairs where the key is the join key. This may involve two additional map transformations. This is not necessary when we join data using DataFrames. 

## Missing Values

Similar to Pandas, DataFrames come equipped with functions to address missing data.
+ `dropna` function: can be used to remove observations with missing values
+ `fillna` function: can be used to fill missing values with a default value

`dropna` function can be used to remove observations where helpful factor is missing.

In [ ]:
dense_products_df = product_df.dropna(subset=["price"])
dense_products_df.show()

`na.drop()` function is an alias to same behaviour within DataFrames API

In [ ]:
dense_products_df_2 = product_df.na.drop(subset=["price"])
dense_products_df_2.show()

`fillna` function can be used to replace a set of missing values with a sensible alternative values (eg: a default) by first figuring out an ideal value as a default value for the helpful field.

In [34]:
from pyspark.sql.functions import mean
average = select_df.select(mean(select_df.helpful))
average.show()

+------------------+
|      avg(helpful)|
+------------------+
|0.7383332842163571|
+------------------+



### Exercise

Use the `select` function with other summarisation functions to summarise values in the table in 3 columns as follows (5 mins):

    1) average helpful score for reviews
    
    2) number of reviews in the dataframe
    
    3) maximum overall score for a review amongst all reviews
    
**Hint: `pyspark.sql.functions` module contains various useful functions**. Full set of functions can be found at http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [ ]:
from pyspark.sql.functions import mean, count, max
summary = select_df.select(mean(select_df.helpful), 
                           count(select_df.asin), 
                           max(select_df.overall))
summary.show()


In [35]:
default_value = 0.7383332842163571
dense_reviews_df = select_df.fillna(default_value, subset=["helpful"])
dense_reviews_df.show()

+----------+-------+--------------------+------------------+--------------+--------------+
|      asin|overall|          reviewText|           helpful|    reviewerID|unixReviewTime|
+----------+-------+--------------------+------------------+--------------+--------------+
|0132793040|    5.0|Corey Barker does...|               1.0| AKM1MP6P0OYPR|    1365811200|
|0321732944|    5.0|While many beginn...|0.7383332842163571|A2CX7LUOHB2NDG|    1341100800|
|0439886341|    1.0|It never worked. ...|               1.0|A2NWSAGRHCP8N5|    1367193600|
|0439886341|    3.0|Some of the funct...|               1.0|A2WNBOD3WNDNKT|    1374451200|
|0439886341|    1.0|Do not waste your...|               1.0|A1GI0U4ZRJA8WN|    1334707200|
|0511189877|    5.0|Dog got the old r...|0.7383332842163571|A1QGNMC6O1VW39|    1397433600|
|0511189877|    2.0|This remote, for ...|               1.0|A3J3BRHTDRFJ2G|    1397433600|
|0511189877|    5.0|We had an old Tim...|               0.0|A2TY0BTJOTENPG|    1395878400|

In [36]:
default_value = .0
dense_reviews_df_2 = select_df.na.fill(default_value, subset=["helpful"])
dense_reviews_df_2.show()

+----------+-------+--------------------+------------------+--------------+--------------+
|      asin|overall|          reviewText|           helpful|    reviewerID|unixReviewTime|
+----------+-------+--------------------+------------------+--------------+--------------+
|0132793040|    5.0|Corey Barker does...|               1.0| AKM1MP6P0OYPR|    1365811200|
|0321732944|    5.0|While many beginn...|               0.0|A2CX7LUOHB2NDG|    1341100800|
|0439886341|    1.0|It never worked. ...|               1.0|A2NWSAGRHCP8N5|    1367193600|
|0439886341|    3.0|Some of the funct...|               1.0|A2WNBOD3WNDNKT|    1374451200|
|0439886341|    1.0|Do not waste your...|               1.0|A1GI0U4ZRJA8WN|    1334707200|
|0511189877|    5.0|Dog got the old r...|               0.0|A1QGNMC6O1VW39|    1397433600|
|0511189877|    2.0|This remote, for ...|               1.0|A3J3BRHTDRFJ2G|    1397433600|
|0511189877|    5.0|We had an old Tim...|               0.0|A2TY0BTJOTENPG|    1395878400|

## Using Spark SQL

Spark DataFrames also allow you to use Spark SQL to query from Petabytes of data. Spark comes with a SQL like query language which can be used to query from Distributed DataFrames. A key advantage of using Spark SQL is the use of [Catalyst query optimizer](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html "Catalyst") and [Project Tungsten](https://databricks.com/session/deep-dive-into-project-tungsten-bringing-spark-closer-to-bare-metal) under the hood to convert your SQL query to run it most efficiently. Spark converts the SQL syntax to a DAG that can be executed to realize the desired result.

### Example Queries

Spark SQL can leverage the same functionality as the DataFrame API provides. In fact, it provides more functionality via SQL capabilities and HQL capabilities that are available to Spark SQL environment. 

Lets look at different functions available in Spark SQL environment by using examples that use multiple functions. This will benefit by:
+ Covering many functions that are possible via spark SQL
+ Giving an understanding about how to pipe multiple functions together


First, we register the dataframes we have in SQLContext to introduce references that we can use to do transformations to the DataFrames.

In [37]:
# Register the DataFrames to be used in sql
dense_reviews_df.createOrReplaceTempView("reviews")
product_df.createOrReplaceTempView("products")

print("There are {0} reviews about {1} products".format(dense_reviews_df.count(),product_df.count()))

There are 30000 reviews about 2469 products


In [38]:
sql_query = """SELECT reviews.asin, overall, reviewText, price
            FROM reviews JOIN products ON reviews.asin=products.asin
            WHERE price > 50.00
"""

result = spark.sql(sql_query)
result.show()

+----------+-------+--------------------+------+
|      asin|overall|          reviewText| price|
+----------+-------+--------------------+------+
|0528881469|    5.0|Love it has every...|299.99|
|0528881469|    1.0|I have owned two ...|299.99|
|0528881469|    5.0|We got this GPS f...|299.99|
|0528881469|    1.0|I'm a professiona...|299.99|
|0528881469|    4.0|This is a great t...|299.99|
|0528881469|    3.0|Well, what can I ...|299.99|
|0528881469|    2.0|Not going to writ...|299.99|
|0528881469|    2.0|My brother is a t...|299.99|
|0528881469|    4.0|This unit is a fa...|299.99|
|0528881469|    5.0|I did a lot of co...|299.99|
|0528881469|    4.0|I purchased this ...|299.99|
|0528881469|    5.0|EXCELLENT. BEST T...|299.99|
|0528881469|    1.0|I was real psyche...|299.99|
|0528881469|    4.0|Well as one of th...|299.99|
|0528881469|    1.0|Thought the unit ...|299.99|
|0528881469|    4.0|Was fast and what...|299.99|
|0528881469|    2.0|Twice this item h...|299.99|
|0528881469|    1.0|

### Exercise

As a data scientist, imagine you hypothesize the following claim: 

*The price of goods and the overall score of reviews are correlated.*

What is a good experiment to quickly sanity check if this is True??

To support this experiment, formulate a SQL query that calculates the average product price score for each distinct overall score value.

In [39]:
sql_query = """SELECT overall, avg(price)
            FROM reviews JOIN products ON reviews.asin=products.asin
            GROUP BY overall
            ORDER BY overall 
"""

result_2 = spark.sql(sql_query)
result_2.show()


+-------+------------------+
|overall|        avg(price)|
+-------+------------------+
|    1.0|47.950229775011934|
|    2.0|47.709516129032274|
|    3.0| 44.33610311750602|
|    4.0|44.286331981874596|
|    5.0| 41.57316541722106|
+-------+------------------+



## User Defined Functions

Spark SQL also provides the functionality similar to User Defined Functions (UDF) offering in Hive. Spark uses `udf.register()` function to register python functions in a Spark Session.

You can register user defined functions with/ without a return type.

In the following example, a python function that uses regex and `str.lower()` function is introduced to SQLContext under the reference `sanitise()`. 

In [40]:
import re

def transform_review(review):
    """ takes a review string and cleans it by
        1) removing non-alphanumeric charactors
        2) turning all alphabetic charactors to lower case
        
        Args:
            review (str): review text
        
        Returns:
            str: sanitised review text
    """
    x1 = re.sub('[^0-9a-zA-Z\s]+', '', review)
    return x1.lower()

result.createOrReplaceTempView("result")

# register the UDF
spark.udf.register("sanitise", transform_review, returnType=StringType())

# use the UDF in SQL 
sql_query_transform = """SELECT asin, reviewText, sanitise(reviewText) as cleaned
            FROM result
"""

result_transform = spark.sql(sql_query_transform)
result_transform.show()

+----------+--------------------+--------------------+
|      asin|          reviewText|             cleaned|
+----------+--------------------+--------------------+
|0528881469|Love it has every...|love it has every...|
|0528881469|I have owned two ...|i have owned two ...|
|0528881469|We got this GPS f...|we got this gps f...|
|0528881469|I'm a professiona...|im a professional...|
|0528881469|This is a great t...|this is a great t...|
|0528881469|Well, what can I ...|well what can i s...|
|0528881469|Not going to writ...|not going to writ...|
|0528881469|My brother is a t...|my brother is a t...|
|0528881469|This unit is a fa...|this unit is a fa...|
|0528881469|I did a lot of co...|i did a lot of co...|
|0528881469|I purchased this ...|i purchased this ...|
|0528881469|EXCELLENT. BEST T...|excellent best tr...|
|0528881469|I was real psyche...|i was real psyche...|
|0528881469|Well as one of th...|well as one of th...|
|0528881469|Thought the unit ...|thought the unit ...|
|052888146

### Exercise

You can see from the data that Categories variable is a list of lists of categories. In the list of categories, each list represents a product category where the collection of strings represent the hierarchical structure of the categories. 

eg: `["Women's Fasion", "Eveningwear", "Dress"]` 

        represents the category  `Women's Fasion --> Eveningwear --> Dress`

In terms of dealing with these categories, it is easier for us to use this data if the category field was a flat list of string where each string is a unique product category

Implement a python function that takes in a list of lists of strings and convert it into a flat list of strings. Every string in this flat list is a **unique** product category where the hierarchy within the category is shown with ` > ` symbol.

eg: `["Women's Fasion", "Eveningwear", "Dress"]` --> `"Women's Fasion > Eveningwear > Dress"`

Now use this function to transform the `categories` field in `product_df` (15 Mins)

In [41]:
SEPERATOR = " > "

def flatten_unique_categories(categories):
    """Takes in a list of categories and returns back the flattened unique set of categories
    assigned to specific product. It does the following:
        1) connects the strings in the hierarchical category using the seperator
        2) generates a list of distinct categories per record
    
    Args:
        categories ([[str]]): a list of categories where every category is a list of strings
        
    Returns:
        flattened_unique_cats ([str]): list of unique categories where the heierarchy within the
                                      category is preserved using a ' > ' charactor                                      
    """
    # 1. join the strings in category heierarchy using the SEPERATOR
    flat_cats = [SEPERATOR.join(cat) for cat in categories]

    # 2. get the unique categories and return them in a list
    flattened_unique_cats = list(set(flat_cats))

    # return the result
    return flattened_unique_cats

Now lets test our function

In [42]:
test = [list("abc"), list("bc"), list("ac"), list("bc")]

actual = flatten_unique_categories(test)

print(actual)

['b > c', 'a > b > c', 'a > c']


Now lets use this function to transform `categories` field in `product_df`

In [43]:
# register the dataframe as a table

product_df.createOrReplaceTempView("products")

# 1. register the UDF

spark.udf.register("flatten_unique", 
                   flatten_unique_categories, 
                   returnType=ArrayType(StringType()))

# 2. use the UDF in SQL 

sql_query_flatten = """SELECT asin, title, price, flatten_unique(categories) as flattened
                        FROM products
                    """

result_flatten = spark.sql(sql_query_flatten)
result_flatten.show(truncate=False)

+----------+---------------------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|asin      |title                                                                                                                            |price|flattened                                                                                                                                                                                                                                                                              |
+----------+---------------------------------------------------------------------------------------------------------------------------------+

In [44]:
a = [1,2,4,5,6,1]
print(a)

[1, 2, 4, 5, 6, 1]


In [46]:
al = list(set(a))
print(al)

[1, 2, 4, 5, 6]
